In [1]:
# !pip install beautifulsoup4

## I. Tổng quan

1) Chuẩn bị categories và trang web cần tải
2) Get link vào trong file .txt
3) Từ file .txt tải ảnh về folder
4) Clean data => folder pro

### 1) Chuẩn bị categories và trang web cần tải

In [2]:
animal  =  ["Monkey",  "Elephant",  "cows",  
"Cat",  "Dog",  "bear",  "fox",  "Civet", 
"Pangolins", "Rabbit", "Bats", "Whale", 
"Cock", "Owl", "flamingo", "Lizard", "Turtle", 
"Snake", "Frog", "Fish", "shrimp", "Crab", "Snail", 
"Coral", "Jellyfish", "Butterfly", "Flies", "Mosquito", 
"Ants", "Cockroaches", "Spider", "scorpion", "tiger", 
"bird",  "horse", "pig", "Alligator" ,"Alpaca" , 
"Anteater", "donkey", "Bee", "Buffalo", "Camel", 
"Caterpillar", "Cheetah", "Chicken",  "Dragonfly", 
"Duck", "panda", "Giraffe"]

plant = ["Bamboo", "Apple", "Apricot", "Banana", "Bean", 
"Wildflower", "Flower", "Mushroom", "Weed", "Fern" , "Reed", 
"Shrub", "Moss", "Grass", "Palm_tree", "Corn", "Tulip", "Rose",
"Clove", "Dogwood", "Durian", "Ferns", "Fig", "Flax", "Frangipani", 
"Lantana", "Hibiscus", "Bougainvillea", "Pea", "Orchid_Tree", "Rangoon_Creeper",
"Jack_fruit", "Cotton_plant", "Cornelian_tree", "Coffee_plant", "Coconut"
, "wheat", "watermelon", "radish", "carrot"]

furniture = ["bed", "cabinet", "chair", "chests", "clock", 
"desks", "table", "Piano", "Bookcase", "Umbrella", "Clothes", 
"cart", "sofa", "ball", "spoon", "Bowl", "fridge", "pan", "book"]

scenery = ["Cliff", "Bay", "Coast", "Mountains", "Forests", 
"Waterbodies", "Lake", "desert", "farmland", "river", "hedges", 
"plain", "sky", "cave", "cloud", "flower_garden", "glacier", 
"grassland", "horizon", "lighthouse", "plateau", "savannah", "valley", "volcano", "waterfall"]

In [3]:
#f"https://www.freepik.com/search?from_query={name}&query={name}&sort=popular&type=photo&page="
#f"https://www.freeimages.com/search/{name}/"

NameError: name 'name' is not defined

### 2) Get link vào trong file .txt

In [5]:
from bs4 import BeautifulSoup # parse HTML
from urllib.parse import urljoin, urlparse # check link có hợp lệ
import urllib.request # tải ảnh về folder
import threading # lập trình multi threading
import time # sleep, tính toán thời gian

In [6]:
def get_all_images(url):
    """
    args:
        - link trang web chứa ảnh
    
    Returns:
        all image URLs on a single `url`
    """
    soup = BeautifulSoup(urllib.request.urlopen(url), "html.parser")
    urls = []
    for img in soup.find_all("img"):
        img_url = img.attrs.get("src")
        
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
        
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(url, img_url)
        # remove URLs like '/hsts-pixel.gif?c=3.2.5'
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        
        # finally, if the url is valid
        if is_valid(img_url):
            urls.append(img_url)
            
    print(".", end = ' ')
    return urls

In [7]:
def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [8]:
name = 'dog'
ipage = 2
url = f"https://www.freepik.com/search?from_query={name}&query={name}&sort=popular&type=photo&page={ipage}"

In [9]:
# lấy tất cả url chứa link ảnh
# get_all_images(url)

### 3) Từ file .txt tải ảnh về folder

In [10]:
class checkTimeThread():
    def __init__(self, nThreads, npage, urltopic):
        self.nThreads = nThreads
        self.ExcuteThread = None
        self.result = []
        self.npage = npage
        self.urltopic = urltopic
    
    # Func target
    def main(self, start, end):
        
        for i in range(start,end):
            try:
                self.result.extend(get_all_images(self.urltopic+str(i)))
            except:
                pass
        
    def __call__(self):
        
        # Create Threads
        threads = []
        
        #======
        n_batch = self.npage//self.nThreads
        for i in range(0, self.npage, n_batch):
            start = i
            end = i + n_batch
           
            if end >= self.npage:
                end = self.npage + 1

            threads.append(threading.Thread(target=self.main, args = (start, end)))
        
        #======
        start = time.time()
        for i in range(self.nThreads):
            threads[i].start()
        for i in range(self.nThreads):
            threads[i].join()
        end = time.time()
    
        self.ExcuteThread = end - start
    
        return self.result
        
    def __str__(self):
        return f"Excute with {self.nThreads} Thread = {self.ExcuteThread} s\n"

In [11]:
nThreads = 3
npage = 6

In [12]:
animal[0]

'Monkey'

In [13]:
for name in animal:
    result = checkTimeThread(min(nThreads, npage//2), npage, 
                         f"https://www.freepik.com/search?from_query={name}&query={name}&sort=popular&type=photo&page=")()
    
    result2 = list(set(result))
    
    print(f"\nanimal_{name}.txt have {len(result2)} images")
    strResult = '\n'.join(result2)
    with open(f"animal_{name}.txt", "w") as f:
        f.write(strResult)

. . . . . . . 
animal_Monkey.txt have 296 images
. . . . . . . 
animal_Elephant.txt have 295 images
. . . . . . . 
animal_cows.txt have 300 images
. . . . . . . 
animal_Cat.txt have 322 images
. . . . . . . 
animal_Dog.txt have 323 images
. . . . . . . 
animal_bear.txt have 286 images
. . . . . . . 
animal_fox.txt have 276 images
. . . . . . . 
animal_Civet.txt have 140 images
. . . . . . . 
animal_Pangolins.txt have 9 images
. . . . . . . 
animal_Rabbit.txt have 288 images
. . . . . . . 
animal_Bats.txt have 299 images
. . . . . . . 
animal_Whale.txt have 282 images
. . . . . . . 
animal_Cock.txt have 213 images
. . . . . . . 
animal_Owl.txt have 287 images
. . . . . . . 
animal_flamingo.txt have 284 images
. . . . . . . 
animal_Lizard.txt have 257 images
. . . . . . . 
animal_Turtle.txt have 219 images
. . . ..  . . 
animal_Snake.txt have 284 images
. . . . . . . 
animal_Frog.txt have 206 images
. . . . . . . 
animal_Fish.txt have 322 images
. . . . . . . 
animal_shrimp.txt have 301 

In [15]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 22 15:37:46 2021

@author: Quang Van
"""

import wget
import threading
import time
import requests
import urllib.request, urllib.error
import random
import os

class DownloadFromUrl():
    def __init__(self, nThreads, urls, destinate_folder):
        self.nThreads = nThreads
        self.urls = urls
        self.n = len(urls)
        self.destinate_folder = destinate_folder
    
    #===== util ===
    def get_url_status(self, url):  # checks status for each url in list urls
        try:
            urllib.request.urlopen(url)
            requests.get(url)
        except Exception as e:
            return False
        else:
            return True
        
    # Func target
    def down_url(self, start, end):

        for i in range(start, end):
            try:
                response=requests.get(self.urls[i])
            except Exception as e:
                continue
            
            # Tránh trùng tên
            a = random.random()
            with open(f"{self.destinate_folder}/{a}.jpg", "wb") as f:
                f.write(response.content)
            print('.', end=" ")         
                 
    def __call__(self):
        try:
            
            threads = []
            
            #=================
            batch = self.n//self.nThreads
            for i in range(0, self.n, batch):
                start = i
                end = i + batch

                if end >= self.n:
                    end = self.n + 1

                threads.append(threading.Thread(target=self.down_url, args = (start, end)))
        
            #=================
            for i in range(self.nThreads):
                threads[i].start()
            for i in range(self.nThreads):
                threads[i].join()

        except Exception as e:
            pass    
       
    def __str__(self):
        return f"Excute with {self.nThreads} Thread = {self.ExcuteThread} s\n"
    
    


In [16]:
l = []
for name_txt in os.listdir():
    if name_txt.endswith(".txt"):
        l.append(name_txt)

In [17]:
len(l)

134

In [18]:
for name_txt in l:
    try:
        print(f"\n {name_txt}")
        with open(f"{name_txt}", "r") as f:
            content = f.readlines()

        content = [x.strip() for x in content] 

        os.makedirs(name_txt[:-4], exist_ok=True)

        numThread = 30
        DownloadFromUrl(min(numThread, len(content)), content, name_txt[:-4])()

    except Exception:
        pass


 animal_Alligator.txt
. . . . . . . . . . . . . . . ..  . . . ..  ..  . . . . . . ..  . . . . . . . . . . ..  . . . . . . . ..  . . . . ..  . . ..  . . . . . . . . . . . . ..  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ..  . ..  . . . . . ..  . . . ..  . ..  . . . . . . . . . . . . . . ..  ..  . . . . . . . . . . . . . . . . . . . . . . . . . ..  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
 animal_Alpaca.txt
. . . . . . . . . ..  . . . . . ..  ..  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .. .  . . . . . . . . . . . . . . . . ..  . . . . . . . . . . . . . . . ..  . . .. .  . . . . . ..  . . . ..  . . ..  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ..  . . . . . .. .  . . . . . . . . . . . . . . . . . . . . . . . ..  . . . . . . . . . . . . ...   . . . . . .. .  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ..  . . . . . . . . . .

In [ ]:
numThread = 10
DownloadFromUrl(min(numThread, len(content)//2), content, "animal_Alpaca")()

### 4) Clean data => folder pro